<a href="https://colab.research.google.com/github/MSDS-Capstone-Project/LLM_FineTuning/blob/main/LoRA_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Loading the Dataset

try: from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset


# dataset = load_dataset("nvidia/OpenMathInstruct-2", split = "train_1M")
dataset = load_dataset("commonsense_qa")
# dataset_small = dataset.select(range(10000))
# dataset_split = dataset_small.train_test_split(test_size=0.1, seed=42)
# train_data, eval_data = dataset_split["train"], dataset_split["test"]
train_data, eval_data = dataset["train"], dataset["validation"]

print(train_data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 9741
})


In [2]:
eval_data.to_pandas().head()

,id,question,question_concept,choices,answerKey
0,1afa02df02c908a558b4036e80242fac,A revolving door is convenient for two directi...,revolving door,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,a7ab086045575bb497933726e4e6ad28,What do people aim to do at work?,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,b8c0a4703079cf661d7261a60a1bcbff,Where would you find magazines along side many...,magazines,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
3,e68fb2448fd74e402aae9982aa76e527,Where are you likely to find a hamburger?,hamburger,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
4,2435de612dd69f2012b9e40d6af4ce38,James was looking for a good place to buy farm...,farmland,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [3]:
### Logging into HuggingFace

try:
  from dotenv import load_dotenv
except:
  !pip install python-dotenv
  from dotenv import load_dotenv

from huggingface_hub import login, whoami
import os

load_dotenv()
login(os.getenv("hugging_face_key2"))

In [4]:
!pip install peft
!pip install -U bitsandbytes
!pip install --upgrade transformers accelerate

In [5]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# import torch

# # model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# model_name = "gpt2"

# # bnb_config = BitsAndBytesConfig(
# #     load_in_4bit=True,
# #     bnb_4bit_compute_dtype=torch.float16,
# #     bnb_4bit_use_double_quant=True,
# #     bnb_4bit_quant_type="nf4"
# # )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     # device_map="auto",
#     # quantization_config=bnb_config,
#     # torch_dtype=torch.float16
# )

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

# ### LoRA Config

# from peft import LoraConfig, get_peft_model

# peft_config = LoraConfig(
#     r=8,  # Rank of the low-rank adaptation - attention heads
#     lora_alpha=32, # alpha scaling
#     lora_dropout=0.1,
#     bias="none",
#     target_modules=["c_attn", "c_proj"]  # GPT-2 specific layers
# )


# peft_model = get_peft_model(model, peft_config)
# peft_model.print_trainable_parameters()


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
import torch

model_name = "gpt2"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.float16
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # quantization_config=bnb_config,
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["c_attn", "c_proj"]
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [7]:
import numpy as np
def format_prompt(example):
    question = example['question']
    choices = example['choices']['text']
    labels = example['choices']['label']
    answer_key = example['answerKey']

    # Find the correct answer text
    answer_index = labels.index(answer_key)
    answer_text = choices[answer_index]

    # Create the prompt
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    full_text = prompt + " " + answer_text

    # Tokenize full input
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Calculate where the completion starts
    prompt_ids = tokenizer(prompt,
                           truncation=True,
                           max_length=128)["input_ids"]
    completion_start = len(prompt_ids)

    # Mask prompt tokens in the labels
    labels = [-np.inf] * completion_start + tokenized["input_ids"][completion_start:]
    labels += [-np.inf] * (128 - len(labels))  # pad label to match input

    tokenized["labels"] = labels
    return tokenized


tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [8]:
tokenized_train.to_pandas().head()

,input_ids,attention_mask,labels
0,"[48, 25, 383, 9388, 1028, 262, 1524, 547, 257,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
1,"[48, 25, 44067, 2227, 284, 467, 284, 810, 262,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
2,"[48, 25, 1675, 17276, 257, 442, 11020, 407, 51...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
3,"[48, 25, 3012, 20347, 290, 584, 12763, 290, 46...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
4,"[48, 25, 383, 21831, 6807, 422, 262, 1748, 656...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."


In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


training_args = TrainingArguments(
    output_dir=f"./IA3_{model_name}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",

    learning_rate=5e-3,

)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)


trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,3.764100,3.234744
200,3.339800,3.174899
300,3.305300,3.183425
400,3.326500,3.160097
500,3.277100,3.173862
600,3.274800,3.134918
700,3.284600,3.125344
800,3.249200,3.102061
900,3.212500,3.074513
1000,3.223500,3.071784


TrainOutput(global_step=3654, training_loss=3.0068998039258132, metrics={'train_runtime': 331.9194, 'train_samples_per_second': 88.042, 'train_steps_per_second': 11.009, 'total_flos': 1927135929434112.0, 'train_loss': 3.0068998039258132, 'epoch': 3.0})

In [10]:
peft_model.save_pretrained(f"./LoRA_{model_name}_final")
tokenizer.save_pretrained(f"./LoRA_{model_name}_final")

('./LoRA_gpt2_final/tokenizer_config.json',
 './LoRA_gpt2_final/special_tokens_map.json',
 './LoRA_gpt2_final/vocab.json',
 './LoRA_gpt2_final/merges.txt',
 './LoRA_gpt2_final/added_tokens.json',
 './LoRA_gpt2_final/tokenizer.json')

In [11]:
from peft import PeftModel, PeftConfig
peft_config = PeftConfig.from_pretrained(f"./LoRA_{model_name}_final")

base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, f"./LoRA_{model_name}_final")

tokenizer = AutoTokenizer.from_pretrained(f"./LoRA_{model_name}_final")

In [13]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Use the PEFT model (already trained) and tokenizer
peft_model.eval()
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)

def format_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    return prompt, choices

correct = 0
total = 0

for example in tqdm(eval_data, total=len(eval_data)):
    prompt, choices = format_prompt(example)

    # Generate prediction
    output = pipe(prompt, max_new_tokens=10, do_sample=False, eos_token_id=tokenizer.eos_token_id)[0]["generated_text"]

    # Extract generated answer (after "Answer:")
    generated = output.split("\n")[2].split("A:")[-1].strip().split(" /// ")[0].strip()


    # Get ground truth
    answer_key = example["answerKey"]
    labels = example["choices"]["label"]
    true_answer = example["choices"]["text"][labels.index(answer_key)].strip()

    print("\n".join(output.split("\n")[:3]))
    print(f"\n\nModel: {generated} and Actual Answer: {true_answer}")

    if generated == true_answer:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Accuracy: {accuracy:.2%}")


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

Q: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?
O: bank /// library /// department store /// mall /// new york
A:  mall store store store store store store store store store


Model: mall store store store store store store store store store and Actual Answer: bank
Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A:  learn from each other's own source of information from


Model: learn from each other's own source of information from and Actual Answer: complete job


  0%|          | 4/1221 [00:00<04:28,  4.54it/s]

Q: Where would you find magazines along side many other printed works?
O: doctor /// bookstore /// market /// train station /// mortuary
A:  library station station station station station station station station station


Model: library station station station station station station station station station and Actual Answer: bookstore
Q: Where are  you likely to find a hamburger?
O: fast food restaurant /// pizza /// ground up dead cows /// mouth /// cow carcus
A:  pizza restaurant carcuscuscus carcus carcus


Model: pizza restaurant carcuscuscus carcus carcus and Actual Answer: fast food restaurant


  0%|          | 6/1221 [00:01<04:11,  4.83it/s]

...

Q: His compressor needed a new hose, where did he go?
O: garden shed /// hardware store /// brothel /// garage /// greenhouse
A:  hardware store store store store store store store store store


Model: hardware store store store store store store store store store and Actual Answer: hardware store
Q: The man closed his eyes as the music played, what effect did the music have?
O: coma /// enjoyable /// soothing /// universal /// good or bad
A:  soothing or bad or bad bad bad or bad or


Model: soothing or bad or bad bad bad or bad or and Actual Answer: soothing


100%|█████████▉| 1220/1221 [03:50<00:00,  5.05it/s]

Q: Setting up framing, truss and beam are some of the first steps in what?
O: new construction /// warehouse /// driving /// ceiling /// bridge
A:  bridge bridge bridge bridge bridge bridge bridge bridge bridge bridge


Model: bridge bridge bridge bridge bridge bridge bridge bridge bridge bridge and Actual Answer: new construction


100%|██████████| 1221/1221 [03:51<00:00,  5.28it/s]

Q: What is another name for a disk for storing information?
O: computer store /// computer to store data /// computer hard drive /// cd player /// usb mouse
A:  computer store data to store information to store information to


Model: computer store data to store information to store information to and Actual Answer: computer hard drive
Accuracy: 3.60%


In [14]:
model.eval()
import torch
# prompt = f"Q: {question}\nChoices:\n"
#     for choice in choices:
#         prompt += f"- {choice}\n"
#     prompt += "Answer:"
q = "What do people aim to do at work?"
options = ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']
options2 = ' /// '.join(options)
from IPython.display import display, Math
prompt = f"Q: {q}\nO: {options2}\nA: "
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)

with torch.no_grad():
    outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=10,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    early_stopping=True,
    num_beams=5,  # Optional: beam search for stronger outputs
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A:  learn from each other, learn from each other,


In [15]:
import math
from transformers import Trainer, TrainingArguments

eval_results = trainer.evaluate(eval_dataset=tokenized_eval)
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 14.31
